# 2D+ MOT Simulation  
Before we move onto simulating our system, let's compare with DOI: 10.1103/PhysRevA.74.023406.  
Physical Dimensions:  
1. glass cell: 16.4cm $\times$ 4cm $\times$ 4cm
2. beam size(horizontal): 96 mm (waist) $\times$ 9 mm (waist) / $\sigma^+ - \sigma^-$ configuration
3. differential tube: 2mm (diameter) for 1cm long / 3mm (diameter) for 3cm long
4. beam size(vertical): 9mm (waist) / linearly polarized

For ${}^{87}Rb$, detuning is $2\Gamma$,but I should double-check the remaining parts. (may differ by measurements)

TODO: add pushing / retarding beam configuration with 2D MOT. evaluate in random 3D scattering process.

In [2]:
# imports
import numpy as np
import pylcp
import matplotlib.pyplot as plt
import scipy.constants as cts
from jinja2 import pass_context


In [3]:
atom = pylcp.atom("87Rb")
# set unitless mass
mass = (atom.state[2].gamma*atom.mass)/(cts.hbar*(100*2*np.pi*atom.transition[1].k)**2)
print(mass)

804.3755599734405


In [6]:
# set hamiltonian
def return_hamiltonian(Fl):
    Hg, Bgq = pylcp.hamiltonians.singleF(F=Fl, gF=0, muB=1)
    He, Beq = pylcp.hamiltonians.singleF(F=Fl+1, gF=1/(Fl+1), muB=1)
    # Check the muB unit. -> we set megneton as 1. convert when we interested physical value (force etc.)
    # How we can evaluate dipole matrix before we introduce the field? -> evaluate all polarizaiotn case
    dijq = pylcp.hamiltonians.dqij_two_bare_hyperfine(Fl,(Fl+1))
    hamiltonian = pylcp.hamiltonian(Hg, He, Bgq, Beq, dijq, mass = mass)
    return hamiltonian

# set horizontal laser beams
laserbeams = pylcp.conventional2DMOTBeams(det=2, s=1)
#TODO: change appropriate quadrupole profile
# magfield
hamiltonian = return_hamiltonian(2)

AttributeError: module 'pylcp' has no attribute 'conventional2DMOTBeams'